# 零样本图像识别方法v2.0（词向量方法）

## 初步数据处理

### 词向量读取处理

In [2]:
#
f = open(r'DatasetB_20180919/class_wordembeddings.txt')
wordembeddings = f.readlines()
class_wordembeddings = []
f.close()
for line in wordembeddings:

    class_wordembeddings.append(line.split())


In [4]:
f = open(r'DatasetB_20180919/label_list.txt')
label_list = f.readlines()
label_lists={}
f.close()
for line in label_list:
    m, n = line.split()
    label_lists[n] = m

### 对lable做一次清洗

In [5]:
#对lable做一次清洗
import numpy as np

import pandas as pd

class_wordembedding = pd.DataFrame(class_wordembeddings)
#将标签ID与30位属性向量对应起来，生成表格形式的数据结构

for i in range(285):
    class_wordembedding.iloc[i][0] = label_lists[class_wordembedding.iloc[i][0]]


class_wordembedding = class_wordembedding.set_index(0)
class_wordembedding=class_wordembedding.astype(float)

for i in range(class_wordembedding.shape[0]):
    for j in range(class_wordembedding.shape[1]):
        if class_wordembedding.iloc[i][j+1] >1:
            class_wordembedding.iloc[i][j+1] =1
        if class_wordembedding.iloc[i][j+1] <-1:
            class_wordembedding.iloc[i][j+1] =-1

class_wordembedding = class_wordembedding.reset_index()  

class_wordembedding.to_csv("class_wordembedding.csv", index=None)


###  生成测用于测试集预测的wordclass

In [48]:
import numpy as np

import random


data_train = open(r'DatasetA_train_20180813/train.txt')

data_train = data_train.readlines()

length = len(data_train)

dic_train={}
#此处跟随样本分类种类设定

for i in range(length):

    m,n = data_train[i].split()
    
    if n not in dic_train:
        
        dic_train[n]=1

In [52]:
#对lable做一次清洗
import numpy as np

import pandas as pd

class_wordembedding = pd.DataFrame(class_wordembeddings)
#将标签ID与30位属性向量对应起来，生成表格形式的数据结构
count=0
for i in range(230-count):
    class_wordembedding.iloc[i-count][0] = label_lists[class_wordembedding.iloc[i-count][0]]
    if class_wordembedding.iloc[i-count][0] in dic_train:
        class_wordembedding.drop(class_wordembedding.index[i-count],inplace=True)
        count =count+1

class_wordembedding = class_wordembedding.set_index(0)
class_wordembedding=class_wordembedding.astype(float)

for i in range(class_wordembedding.shape[0]):
    for j in range(class_wordembedding.shape[1]):
        if class_wordembedding.iloc[i][j+1] >1:
            class_wordembedding.iloc[i][j+1] =1
        if class_wordembedding.iloc[i][j+1] <-1:
            class_wordembedding.iloc[i][j+1] =-1

class_wordembedding = class_wordembedding.reset_index()  

class_wordembedding.to_csv("test_wordembedding.csv", index=None)


### 导入训练集数据

In [ ]:
#导入一些之后需要的参数
from keras.preprocessing import image

import pandas as pd

import numpy as np

import random

data_atten = pd.read_csv(r"class_wordembedding.csv")

data_atten = data_atten.set_index('0')
# 通过set_index方法，可以设置单索引和复合索引。

data_train = open(r'DatasetA_train_20180813/train.txt')

data_train = data_train.readlines()

random.shuffle(data_train)
#此处为了加快速度只输入了10000张样本
data_train=data_train[:10000]
path = r'DatasetA_train_20180813/train/'

length = len(data_train)

train_x = np.zeros((length, 64, 64, 3))

train_y = np.zeros((length, 300))
#此处跟随样本分类种类设定

for i in range(length):

    m,n = data_train[i].split()

    img = image.load_img(path + m)

    
    train_x[i] = image.img_to_array(img) /255

    train_y[i] = data_atten.loc[n]

## 模型建立与训练

### 建立模型

In [34]:
#build model

from keras import Model, Sequential

from keras.applications import VGG16, VGG19

from keras.layers import Flatten, Dense



from sklearn.model_selection import train_test_split

from keras.optimizers import *

from densenet121 import DenseNet


base_model =  DenseNet()


x = base_model.output

x = Flatten()(x)

x = Dense(1024, activation='relu')(x)
#全连接层
predictions = Dense(300, activation='linear', name="wordembedding")(x) 

 

model = Model(inputs=base_model.input, outputs= predictions)

optimizer = SGD(lr=0.01, momentum=0., decay=0., nesterov=False)
# #此处可修改优化器，在model.compile中optimizer修正
# # optimizer = RMSprop(lr=0.001,rho=0.9,epsilon=1e-06)

model.compile(optimizer=optimizer, loss='cosine_proximity')

#loss可尝试使用mse
model.summary()


C:\Users\tree\PycharmProjects\albb\densenet121.py:45: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (7, 7), strides=(2, 2), name="conv1", use_bias=False)`
  x = Convolution2D(nb_filter, 7, 7, subsample=(2, 2), name='conv1', bias=False)(x)
C:\Users\tree\PycharmProjects\albb\densenet121.py:99: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (1, 1), name="conv2_1_x1", use_bias=False)`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base+'_x1', bias=False)(x)
C:\Users\tree\PycharmProjects\albb\densenet121.py:109: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), name="conv2_1_x2", use_bias=False)`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base+'_x2', bias=False)(x)


TypeError: 'module' object is not callable

### 模型训练

In [94]:
#  ```python
#训练模型
h = model.fit(train_x, {'wordembedding':train_y[:,:]},epochs=200,validation_split=0)

#train_y对应所选择输入利用的标签，
#validation_split预留样本做验的比例，可以设定较小一点
#  ```
 

Epoch 1/200
1000/1000 [==============================] - 13s 13ms/step - loss: -0.2458
Epoch 2/200
1000/1000 [==============================] - 3s 3ms/step - loss: -0.2711
Epoch 3/200
1000/1000 [==============================] - 3s 3ms/step - loss: -0.2712
Epoch 4/200
1000/1000 [==============================] - 3s 3ms/step - loss: -0.2716
Epoch 5/200
1000/1000 [==============================] - 3s 3ms/step - loss: -0.2718
Epoch 6/200
1000/1000 [==============================] - 3s 3ms/step - loss: -0.2723
Epoch 7/200
1000/1000 [==============================] - 3s 3ms/step - loss: -0.2720
Epoch 8/200
1000/1000 [==============================] - 3s 3ms/step - loss: -0.2725
Epoch 9/200
 192/1000 [====>.........................] - ETA: 2s - loss: -0.2741

KeyboardInterrupt: 

## 测试与分析

### 导入模型参数直接进行预测（用于在windows上测试导入模型）

In [16]:
import pandas as pd

import numpy as np

from keras.models import load_model

from keras.preprocessing import image
 
model = load_model('0921/model_epoch_3.h5')
#导入目录下的模型文件并作出预测
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0         
__________

## 直接输出结果输出结果

In [2]:
data_atten = pd.read_csv(r"class_wordembedding.csv")
IDs= []
for i in range(40):
    IDs.append(data_atten.loc[i][0])
data_atten = data_atten.set_index('0')

NameError: name 'pd' is not defined

In [58]:
data_test = open(r"DatasetA_test_20180813/DatasetA_test/image.txt")

data_test = data_test.readlines()

path_test = r'DatasetA_test_20180813/DatasetA_test/test/'

length = len(data_test)

data_test_x = np.zeros((length, 64, 64, 3))

for i in range(length):

    m = data_test[i][:-1]

    img = image.load_img(path_test + m)

    data_test_x[i] = image.img_to_array(img)/255
    
res = model.predict(data_test_x)


In [61]:
#遍历标签ID与对应属性值
data_atten_res = []

dic={}

count = 0

length = len(res[0])

for ID in IDs:
    key=ID
    print(key)
    data_atten_res.append(np.hstack((data_atten.loc[key])))
    
    dic[count] = key

    count = count+1





ZJL210
ZJL205
ZJL207
ZJL206
ZJL209
ZJL202
ZJL203
ZJL201
ZJL204
ZJL208
ZJL223
ZJL214
ZJL240
ZJL238
ZJL237
ZJL228
ZJL220
ZJL212
ZJL213
ZJL221
ZJL239
ZJL217
ZJL236
ZJL219
ZJL216
ZJL211
ZJL230
ZJL226
ZJL229
ZJL234
ZJL232
ZJL233
ZJL227
ZJL222
ZJL231
ZJL218
ZJL215
ZJL225
ZJL235
ZJL224


In [65]:
#将预测输出进行处理，组合相对应的
vec_result = []#np.zores((len(res[0]), 18))

for i in range(res.shape[0]):

    vec_result.append(res[i])

In [70]:
# #将预测输出进行处理，组合相对应的
# vec_result = []#np.zores((len(res[0]), 18))

# for i in range(res.shape[0]):

#     vec_result.append(res[i])

dis_list = []


# accury=0
file  = open("0914/result0914out1.txt",'a+')
# file  = open("retaines_test.txt",'a+')
for i in range(len(vec_result)):
    dis_list=[]
    m= data_test[i]
#     m= data_test[i].split()
    for j in range(len(data_atten_res)):

        vec1 = vec_result[i]
        
        vec2 = data_atten_res[j]
        
        cos_theta_1 = np.sum(np.multiply(vec1, vec2))
        
        cos_theta_2 = np.linalg.norm(vec1) * np.linalg.norm(vec2)
        
        dist = cos_theta_1/cos_theta_2

        dis_list.append(dist)
    m = data_test[i][:-1]
    print(m)
    print("witch is : "+ dic[dis_list.index(max(dis_list))] + " and max dis = " + str(max(dis_list)))
#     if n == dic[dis_list.index(max(dis_list))]:
#         accury += 1
#         print('true')    
    file.write(m+'\t'+dic[dis_list.index(max(dis_list))]+'\n')
file.close()

00ddbe75d7aff5037d360401af02ca57.jpg
witch is : ZJL225 and max dis = 0.4134826151462185
03357448dd6ce1ac0b5b69e9204c53ee.jpg
witch is : ZJL215 and max dis = 0.5579509097646094
051b37cb697a5dbd6230fa9b58bc170d.jpg
witch is : ZJL206 and max dis = 0.29124752106479096
08247ec84ef66743ac30ba51cf2c1607.jpg
witch is : ZJL236 and max dis = 0.3979665388981931
093e28418698ca07ca805377e4533f96.jpg
witch is : ZJL211 and max dis = 0.4734635432291129
0dc47ee8e5804b2fc35e453c7a5fec2d.jpg
witch is : ZJL204 and max dis = 0.5199495281525557
145b317ddf3abf424eec3aeded7e8af4.jpg
witch is : ZJL215 and max dis = 0.5614727589587766
14ffb0bdf864e72dddef53ae1e8933be.jpg
witch is : ZJL216 and max dis = 0.3428839825089239
156301bb40a3b8034aca3443a3a5a345.jpg
witch is : ZJL240 and max dis = 0.5165048469718762
183fb67145085b6f490362651225e539.jpg
witch is : ZJL204 and max dis = 0.40885465433793905
18c946fa057cf2ffac9bb64b57baab25.jpg
witch is : ZJL240 and max dis = 0.35162664032057556
194376795193972a08ac139d433fb

1b06a592ef5bea10f4e27eef14a51bcc.jpg
witch is : ZJL204 and max dis = 0.49397282358733247
1bfadfd1c7895fe7ce23757f4b39b76e.jpg
witch is : ZJL240 and max dis = 0.4053554021018343
1c92470dcd8bd93eebcfaf8cd09ee89a.jpg
witch is : ZJL221 and max dis = 0.22357217662765025
1db2262d8febfd56ddd7a016f9fc0f18.jpg
witch is : ZJL207 and max dis = 0.4950242572969734
1e61ef72c6fdf333777a28c2a0815734.jpg
witch is : ZJL219 and max dis = 0.5117952089227709
1e7f62e62720f58d7e78a85f5edb4bbf.jpg
witch is : ZJL221 and max dis = 0.292604803444547
1ec36283d552e9aff35948d1feef5367.jpg
witch is : ZJL240 and max dis = 0.5790746321595236
1f43fcf33b49d9466b37de71566d8ce7.jpg
witch is : ZJL215 and max dis = 0.3788565661141859
1fb6f239cb5347a61c1467c1c3e8c2e1.jpg
witch is : ZJL216 and max dis = 0.393289960285741
21b006ae1d824fa1857fa593df8ff2f0.jpg
witch is : ZJL215 and max dis = 0.3785752523078851
21e8eb4a68279349859244bd58c89439.jpg
witch is : ZJL240 and max dis = 0.6040917270722549
2223cc8df556e6ab764ba80a8ba2e0be

7808844b1851c6578619d36c51cfdf48.jpg
witch is : ZJL225 and max dis = 0.3822278033201634
791d6984edca706647780812349b99c6.jpg
witch is : ZJL202 and max dis = 0.4907589844002667
79e6f2c09afce3367a20e70febb0a347.jpg
witch is : ZJL240 and max dis = 0.5724783965845582
7a4293d16ef3db69a9b50bdae06ef32a.jpg
witch is : ZJL216 and max dis = 0.3935013777324205
7b6468daa2e85f4d9fa84ecc3424d717.jpg
witch is : ZJL215 and max dis = 0.37185274487510245
7c33a6ce16025cfbf74f526dcb3768e2.jpg
witch is : ZJL236 and max dis = 0.3911450885990239
7c8503fc6afd33272624be66f742e575.jpg
witch is : ZJL225 and max dis = 0.33282058483024135
7db080d7311b0434e80d1ea17f6e8bce.jpg
witch is : ZJL216 and max dis = 0.4371682972245694
8007e43502d19bdb0feb03d12d66bf5f.jpg
witch is : ZJL216 and max dis = 0.3507937850763422
804b749b8a4fa0d10f3bbeae47a1b735.jpg
witch is : ZJL240 and max dis = 0.5902225596263334
80d55a352b985e01b2618bc5e075780b.jpg
witch is : ZJL216 and max dis = 0.5578099405650564
8159515a877cb985a65131b38ac75e

cf7e945679258a04584ba8720cdafa71.jpg
witch is : ZJL203 and max dis = 0.47530030537181517
cfb240f2d111f4776f8d7d2210baa422.jpg
witch is : ZJL216 and max dis = 0.5396156692762553
d0c7caae7d1751ca8ab4e8f3421bfce8.jpg
witch is : ZJL218 and max dis = 0.5092188560360409
d0e84fd3e2a9de09a4bf3b5aee618c9b.jpg
witch is : ZJL225 and max dis = 0.394963179631754
d0f8bcf8dd0909685577e907c0f72952.jpg
witch is : ZJL217 and max dis = 0.3854561522739268
d1f0b98c2d8af7425c17df0d7bc2537f.jpg
witch is : ZJL207 and max dis = 0.5606416878369436
d251f2e8a89e88c788bcf934a40e05aa.jpg
witch is : ZJL240 and max dis = 0.5384233391841661
d302b0e69445b9f3f59b6d27e61be85d.jpg
witch is : ZJL236 and max dis = 0.3901152397742741
d3b4741fd1f354ee321c3b9d0cbde9e1.jpg
witch is : ZJL206 and max dis = 0.3072315910652051
d3ec5bba697c1c909f2ffb920994e58b.jpg
witch is : ZJL216 and max dis = 0.5697863348653059
d4b38ffd93661052a47a5d410c5766d4.jpg
witch is : ZJL220 and max dis = 0.41511115096920986
d50e529b2704310f3a380e9a9bbb811

3e71d95f31d848a036cd73928515d560.jpg
witch is : ZJL207 and max dis = 0.3642947944833339
3f3b87b37c78c418396bb5508de66a8e.jpg
witch is : ZJL240 and max dis = 0.4769703985237061
400faacbbe46469c3f65e398094fe853.jpg
witch is : ZJL240 and max dis = 0.6026274379902289
4145924b48a0ea5151477a2f8440f46c.jpg
witch is : ZJL207 and max dis = 0.23634912267535613
4171a804d36469d5054a664cf91657e0.jpg
witch is : ZJL216 and max dis = 0.4356030767961466
418f5f8d94f0ab7c5b2102951fecdf51.jpg
witch is : ZJL225 and max dis = 0.39408897061668463
41c66868bd9192819dc8db5d4487f21a.jpg
witch is : ZJL211 and max dis = 0.464161036777293
429545af555e57572230107e15fb8c9b.jpg
witch is : ZJL240 and max dis = 0.6393252699844565
433252821ea1ee78699bc121d03c42db.jpg
witch is : ZJL216 and max dis = 0.3108960541490476
43c0c22039990e020adafa711ccb1de7.jpg
witch is : ZJL204 and max dis = 0.5571187986251833
45c1d0b3f7341c0ef05ba6fd94080ab7.jpg
witch is : ZJL240 and max dis = 0.45493197257647733
46f3c2fede4d3f7efcb71056b72551

...

9e338613b1f49965e91d85a97a0f346c.jpg
witch is : ZJL219 and max dis = 0.3514865062778289
9e617bdedf46a1d8375534c4da2b8fe8.jpg
witch is : ZJL220 and max dis = 0.20668101632881833
9e9550c45d6c95dcc59c03d3047bb419.jpg
witch is : ZJL225 and max dis = 0.3743615533062976
9ed17cdb846b9296375932911f74d3b3.jpg
witch is : ZJL225 and max dis = 0.40496500274209174
9ed201ac752bdde06689b26a0d64b18a.jpg
witch is : ZJL219 and max dis = 0.36870436200985957
9f1b9d18228013eb5d1295f8ce737529.jpg
witch is : ZJL219 and max dis = 0.4769814227899019
9f6757b53d6c7e2c145bd0ecc17ce086.jpg
witch is : ZJL225 and max dis = 0.34473136735926097
9f891d3ce98e45ecd58b80261c0b4988.jpg
witch is : ZJL227 and max dis = 0.2908391559816775
9f9776b999a681f7679da4f365a0e3bb.jpg
witch is : ZJL220 and max dis = 0.4716408930211661
9fceee5d5e5380cc82f928f61e185d16.jpg
witch is : ZJL216 and max dis = 0.4769561451883119
9ff051afe3c5011c9b1032038666955c.jpg
witch is : ZJL233 and max dis = 0.28407944533914525
a06b5a74d37ee55830e3e4db45d

4e86f1ac8d80e632eaed8d3870b7a7ed.jpg
witch is : ZJL227 and max dis = 0.31081152411070007
4eeab2484070e79b4fe48d396db49a4a.jpg
witch is : ZJL225 and max dis = 0.3775824447962899
4f4e75b74c2c79b70caab85535d85a64.jpg
witch is : ZJL236 and max dis = 0.3243679995923522
50395a582134670c3bd6921e2dad9a6b.jpg
witch is : ZJL220 and max dis = 0.37907143304430013
5072588689c27688196432fe61fd46e0.jpg
witch is : ZJL221 and max dis = 0.3119219892740055
50ce8a77dbe204fd34eb83ba95f746ce.jpg
witch is : ZJL219 and max dis = 0.40739896533213465
51ab874a2a50145dbcea68f4236d0ae8.jpg
witch is : ZJL221 and max dis = 0.3062355065627556
544102e38046e9b1664a1767b39935af.jpg
witch is : ZJL215 and max dis = 0.3989479417858955
55d6abe30d9f314fa97b56f075d56ed2.jpg
witch is : ZJL225 and max dis = 0.4122491757637203
56dd2ddbbdf850d6ebe43d0baaf0c2fd.jpg
witch is : ZJL236 and max dis = 0.3575043603975594
586e4b5e95bedec336e226b9334f81f7.jpg
witch is : ZJL203 and max dis = 0.4157135365340299
5892150f799ae75dac2cfc1bbfb46

1d574c8a2232d784030f9f7aa48874f1.jpg
witch is : ZJL216 and max dis = 0.32004897035662644
1e0509e8ec29a4eee9515a297295bd84.jpg
witch is : ZJL236 and max dis = 0.34036131423892624
1f0fb6da73075fc726666b4bb2c95a80.jpg
witch is : ZJL225 and max dis = 0.3471961710580944
1f81bc89804145ff247f5ab78a532272.jpg
witch is : ZJL236 and max dis = 0.345670581459739
1fadb47b0a89095845ea7bdfa4df8d67.jpg
witch is : ZJL216 and max dis = 0.24840346218536855
200ccf9c008900c02d2f73105e28b763.jpg
witch is : ZJL216 and max dis = 0.4293076656693052
213a0bed2103e3b3b209ce9a896182b2.jpg
witch is : ZJL216 and max dis = 0.4070248434262257
23a2f1befc2078ca562b1463b514d628.jpg
witch is : ZJL216 and max dis = 0.36949419581603615
25a4dd0ab2c9b0dff5c75c8af5071b45.jpg
witch is : ZJL227 and max dis = 0.33704659521033226
26298e62aee4b0c3dcaf3826d311d0b5.jpg
witch is : ZJL225 and max dis = 0.31839029325799445
277eb6527acb7b2d949ec4733ae0c35c.jpg
witch is : ZJL215 and max dis = 0.48542422772354776
282c6732d12b5effa279eeb449

witch is : ZJL216 and max dis = 0.25598001685753646
a548ecc769800fe818643ef91d00d213.jpg
witch is : ZJL225 and max dis = 0.4217092495934871
a653eb56624702529b43d0128c1cf723.jpg
witch is : ZJL225 and max dis = 0.4649624966674648
a7bd183726967a9f4ddad23366c39672.jpg
witch is : ZJL225 and max dis = 0.34623589249562886
a80b04438b5db73d14823fbf18a34649.jpg
witch is : ZJL216 and max dis = 0.3102689861087418
a9197408a8d51cfc31da140f476042f4.jpg
witch is : ZJL216 and max dis = 0.3366358141655201
aaa6f30f722b66e1c3d5e7521533f733.jpg
witch is : ZJL216 and max dis = 0.2212656140492637
aadd20cb9a9440d0f9e81960fe2db70e.jpg
witch is : ZJL206 and max dis = 0.3514739728437393
ab0d6add88cf9cffd6c9e584694e10a9.jpg
witch is : ZJL215 and max dis = 0.3298500486562738
ac2206d9aaca7d71db80db83fee08a40.jpg
witch is : ZJL215 and max dis = 0.3052325632068297
ac661a8f8f3a47dee22460c8769768ab.jpg
witch is : ZJL227 and max dis = 0.36167555920761674
acbd8edad421767b1f7eeb2f748bd3a4.jpg
witch is : ZJL216 and max dis

witch is : ZJL236 and max dis = 0.30583887663409226
8191c8e7e230faa10f23c4705b2852ea.jpg
witch is : ZJL220 and max dis = 0.3321137661748547
47043ba514410ad49fd90538a3e45b5e.jpg
witch is : ZJL207 and max dis = 0.3364934041063084
07407854719457e5064e96e50b37d9bb.jpg
witch is : ZJL229 and max dis = 0.3240204952420457
bae95658136ab797fce60436556ac521.jpg
witch is : ZJL216 and max dis = 0.3116878510127304
1a48991334a346a4ad835f1dbcc85f79.jpg
witch is : ZJL236 and max dis = 0.311014182787834
cc114166d86ee0450e17e956afc91cc7.jpg
witch is : ZJL221 and max dis = 0.2835486589860644
13ab063942692fa02415c025c36d884f.jpg
witch is : ZJL216 and max dis = 0.2853303551262144
c9d5d09d6e0e2c08daaaffdc96f12f1d.jpg
witch is : ZJL236 and max dis = 0.38729190538939146
02e02afa2eca17689f00dbeabf8ff588.jpg
witch is : ZJL236 and max dis = 0.38619455034743727
b00dee3c7dcaacb1ccc05b568cf7bf39.jpg
witch is : ZJL216 and max dis = 0.2937096520096067
9e83c1e817d6ce43938f7b12cd87d258.jpg
witch is : ZJL233 and max dis 

## 提取部分训练集做测试

In [17]:
import random

file = open(r'DatasetB_20180919/train.txt')

data_retained_test = file.readlines()

random.shuffle(data_retained_test)

data_retained_test = data_retained_test[:10000]

file.close()

data_atten = pd.read_csv(r"class_wordembedding.csv")

IDs= []
for i in range(40):
    IDs.append(data_atten.loc[i][0])

data_atten = data_atten.set_index('0')

path = r'DatasetB_20180919/train/'

length = len(data_retained_test)

retained_test_x = np.zeros((length, 64, 64, 3))

retained_test_y = np.zeros((length, 300))
#此处跟随更改

data_atten_res = []

dic={}

count = 0

for ID in IDs:

    key = ID
#     print(key)
    
    data_atten_res.append(np.hstack(data_atten.loc[key]))
    
    dic[count] = key

    count = count+1




for i in range(length):

    m,n = data_retained_test[i].split()

    img = image.load_img(path + m)

    
    retained_test_x[i] = image.img_to_array(img)

    retained_test_y[i] = data_atten.loc[n]

retained_test_x=retained_test_x/255

res = model.predict(retained_test_x)

In [18]:
res.shape[0]

10000

In [19]:
#将预测输出进行处理，组合相对应的
vec_result = []#np.zores((len(res[0]), 18))

for i in range(res.shape[0]):

    vec_result.append(res[i])

length = len(vec_result)



In [20]:
# file  = open("retaines_test.txt",'a+')
accury = 0 
for i in range(length):
    dis_list=[]
    m,n = data_retained_test[i].split()
    for j in range(len(data_atten_res)):

        vec1 = vec_result[i]
        
        vec2 = data_atten_res[j]
        
        cos_theta_1 = np.sum(np.multiply(vec1, vec2))
        
        cos_theta_2 = np.linalg.norm(vec1) * np.linalg.norm(vec2)
        
        dist = cos_theta_1/cos_theta_2

        dis_list.append(dist)
        
    print(m)
    print("witch is : "+ dic[dis_list.index(max(dis_list))] + " and min dis = " + str(min(dis_list)))
    if n == dic[dis_list.index(max(dis_list))]:
        accury += 1
        print('true')

#     file.write(m+'\t'+dic[dis_list.index(min(dis_list))]+'\n')
# file.close()

4c51e9ff3a9ad887dfdaf44934b408fb.jpeg
witch is : ZJL41 and min dis = -0.025688473198354413
f7e80ce36d3c953926a1cb2d27b1ff7b.jpeg
witch is : ZJL143 and min dis = -0.05890009594487156
1deb40689d8c9df62f6346eca4eb3729.jpeg
witch is : ZJL83 and min dis = 0.06483263527696435
9c2b6032ef682d079f156c65adf84736.jpeg
witch is : ZJL83 and min dis = -0.16293774257709814
8fb45a518da714d7dc503088b5306955.jpeg
witch is : ZJL83 and min dis = -0.21169192937160838
2dd6ad29fd9813b863263518c3be6e6f.jpeg
witch is : ZJL240 and min dis = -0.15925536739498192
ff17d6bd24f8f7ff7b77e7eaad623092.jpeg
witch is : ZJL36 and min dis = -0.16852430679788755
27ed9e0ac02fd302daed569d410f3e39.jpeg
witch is : ZJL174 and min dis = -0.12708174294866162
6923757cd634507ed314dedddd6002f2.jpeg
witch is : ZJL187 and min dis = -0.13262398664004815
5df9eff3ab19246b1678324a08c19e77.jpeg
witch is : ZJL36 and min dis = -0.21773072935274634
86b7e236093b9c29df48733d03fc08e0.jpeg
witch is : ZJL88 and min dis = -0.20789372417146634
9ff188

365b8ca709367c2e44becdc55ea20b4f.jpeg
witch is : ZJL178 and min dis = -0.06428075545072807
c221f09a362e3bda50dbaf5d015233bd.jpeg
witch is : ZJL207 and min dis = -0.24324401169427778
cf7c95a33ceca6279ebd84f5f715b2f6.jpeg
witch is : ZJL158 and min dis = -0.15520160774097355
09532befe94ceac7db832a2c68e82d47.jpeg
witch is : ZJL153 and min dis = -0.04767213035546646
true
62a7b7a0c1f42ca4167dee3f2c97366c.jpeg
witch is : ZJL240 and min dis = -0.04877957147708525
3a5ca14cb245c07d3395df9159524802.jpeg
witch is : ZJL83 and min dis = -0.1959618548886552
70a16176bd3dbfb0cd748d0b2274ac39.jpeg
witch is : ZJL143 and min dis = -0.08316524431230914
e66e30d4891edfe8095aebce57416212.jpeg
witch is : ZJL87 and min dis = -0.2074869165332121
5428a1482b48010dc568f0171d3385e8.jpeg
witch is : ZJL240 and min dis = -0.24282655360521593
c7a48fc839fd87c3caa1de56bed9b8e9.jpeg
witch is : ZJL187 and min dis = -0.15254367618645998
e9b861a1286b90a889c7072f67dc1518.jpeg
witch is : ZJL68 and min dis = -0.02517222427358231

97a0ab1a9e000c8adcd983cabb9f3852.jpeg
witch is : ZJL104 and min dis = -0.11506835471395618
24dab39fd66d066cd44a5bea3e02ef4f.jpeg
witch is : ZJL38 and min dis = -0.06320507307342141
e24eddf70c6afe6105ac02eaf3760b73.jpeg
witch is : ZJL240 and min dis = -0.08053928931986351
4e817f5d2016b1894975a8a428d678e0.jpeg
witch is : ZJL83 and min dis = -0.07740435102776616
72d79d4e4b8670526ac528bffcdca645.jpeg
witch is : ZJL187 and min dis = -0.1323202928358245
15d6567ec0f0d63683b0b5a448dbdbb8.jpeg
witch is : ZJL207 and min dis = -0.24640293237525335
5f53c21d56d60cd96a5c08dbc32655d4.jpeg
witch is : ZJL143 and min dis = -0.1989330664548786
98c10f28f6fb037fe0d7d1a110566009.jpeg
witch is : ZJL220 and min dis = -0.0931272157841254
594667e71e5399017c07b682d0fe2cc3.jpeg
witch is : ZJL41 and min dis = -0.17216844534154577
883b81978b46fa4b42516ef72aa41223.jpeg
witch is : ZJL87 and min dis = -0.1032000885955244
2dc0080266a10c07e2f5e282bb0f6570.jpeg
witch is : ZJL36 and min dis = -0.0854527233813144
a6a053f02

6b6941bd37f36c1bcbd510830be5114b.jpeg
witch is : ZJL36 and min dis = -0.13787706708931963
172cc9df7f1a182dac84f7046d2ced84.jpeg
witch is : ZJL220 and min dis = -0.06140821911855855
eecea676fa754571d8ca64320065e6b9.jpeg
witch is : ZJL276 and min dis = -0.15299275334880397
f5e5c5f883e63e8f645c2af4d771eff1.jpeg
witch is : ZJL153 and min dis = -0.21946418678745935
84194ebedcf1de67496e1c1b4290ecc6.jpeg
witch is : ZJL41 and min dis = -0.12054689006902951
78ce9cb3bf6ce4b8372659ac97dc4b58.jpeg
witch is : ZJL240 and min dis = -0.19264449826223817
10eb1fa252b6065d29c91e8697ce5804.jpeg
witch is : ZJL240 and min dis = -0.062431491730204056
7f4f74761af83df65f395b43b468e7e2.jpeg
witch is : ZJL207 and min dis = -0.22536298605758057
c74062987327dcc1995dd5eb55e1daef.jpeg
witch is : ZJL207 and min dis = -0.22585957696846545
c9f847a5e028d65db43650a8269c7dd8.jpeg
witch is : ZJL240 and min dis = -0.17082235901768633
8bee1961a947494b6a44acd040749568.jpeg
witch is : ZJL143 and min dis = -0.10326799195003065


d464ffb102f99170627d62811f156b60.jpeg
witch is : ZJL220 and min dis = -0.070953737466163
d3cb6513f336f7981165eb4be1001bc8.jpeg
witch is : ZJL104 and min dis = -0.12465555832527041
0c854e0f87ebef3977533a820d8037e2.jpeg
witch is : ZJL276 and min dis = -0.19854333850169079
20600f2d2bf64047808d5b77a2cf4a86.jpeg
witch is : ZJL99 and min dis = -0.12283899994494753
true
463e8b5f4969c23886f66296036d29fa.jpeg
witch is : ZJL240 and min dis = -0.08297608232540465
d5980e4dae4ac4f0cf38183a1d73364f.jpeg
witch is : ZJL99 and min dis = -0.14153705818771417
11ec5ae6785d2df231404b96b2bb793a.jpeg
witch is : ZJL41 and min dis = -0.14566473717036504
c9bedba817d300e4d93a2619cf9e4b52.jpeg
witch is : ZJL276 and min dis = -0.14012616849501736
6e7836c60b8eaf1f31a8dec1f016a249.jpeg
witch is : ZJL69 and min dis = -0.05303031417094228
true
d24630dc1831203208b5cd5ad3c6e207.jpeg
witch is : ZJL36 and min dis = -0.09904306416121819
24e86b5d894be85b0033c6b7c684911f.jpeg
witch is : ZJL276 and min dis = -0.11857866668631

...

witch is : ZJL89 and min dis = -0.0938743430778211
d8ba56299b0b0436be5cb356c4f7d617.jpeg
witch is : ZJL276 and min dis = -0.18794215363176167
aa46c0c2f255cc098f94d2977cf8ae49.jpeg
witch is : ZJL174 and min dis = -0.15953310804733434
5476460bdb44b065fe2e0a3dc1f1769f.jpeg
witch is : ZJL87 and min dis = -0.2111728293260488
c7979224c4513c9aa85a50b436f3a25c.jpeg
witch is : ZJL38 and min dis = -0.004297768887377239
true
984de51a1ab4c257c6a3ff1084d8a6f3.jpeg
witch is : ZJL88 and min dis = -0.20992315977960904
4e9fae1d90dc1f4a6286707a04a46d2b.jpeg
witch is : ZJL153 and min dis = -0.07106123995166543
true
36d1f2348355a319c6900cc36162788d.jpeg
witch is : ZJL240 and min dis = -0.20216696882277688
3fe47cb08c53729a9ec73d4d844357d1.jpeg
witch is : ZJL158 and min dis = -0.07412331474380739
8f6c827e01725f7145e4ac65d2c99712.jpeg
witch is : ZJL207 and min dis = -0.25630891478554213
6dadac3dd010597c49c6261368e20ac1.jpeg
witch is : ZJL41 and min dis = -0.3692007989252694
c36a39065f4a51dce8459d1783aff92f.j

25044e73f7744cff3268e80b9ecbcd28.jpeg
witch is : ZJL41 and min dis = -0.2029876899417493
e948f67aa4a01e25a130cc1269ec32a6.jpeg
witch is : ZJL87 and min dis = 0.002647746686535021
be9e2c739e10af6b0c645b1fb5bcec16.jpeg
witch is : ZJL68 and min dis = -0.033042556819077945
3b76b43404f8dfb1d4c5d50ac58269c3.jpeg
witch is : ZJL207 and min dis = -0.1966606577775807
587804e1c9cdb5713137e5cb77db7d27.jpeg
witch is : ZJL174 and min dis = -0.14723689248093907
d2c6d54dd6d47a4d27f5f32aa42467b7.jpeg
witch is : ZJL158 and min dis = 0.0008910309081515096
true
72ff67b29b59f1a55402d621e23fd9d5.jpeg
witch is : ZJL68 and min dis = -0.0011316803803276278
5e6d3312ca3e9e41ccb523ca4cee45a6.jpeg
witch is : ZJL144 and min dis = -0.014841977260389994
true
b3dbb6fdf0ac5aff73ef86d5fc34ad8c.jpeg
witch is : ZJL158 and min dis = -0.048707257654792495
ddb8a7c0d64fc244b437877e6beb7821.jpeg
witch is : ZJL240 and min dis = -0.11748324712245088
86eb38c68c299cd3b18816e5f4af3282.jpeg
witch is : ZJL104 and min dis = -0.1469589

9be4ed409137ca7c8ddb9331e40ca7f1.jpeg
witch is : ZJL158 and min dis = -0.020764186696208112
54d476a2a1dce9ea5b4ae6dc362d0381.jpeg
witch is : ZJL158 and min dis = -0.057550416544632436
4e136922f0f556963df2cee3dfce65c0.jpeg
witch is : ZJL228 and min dis = -0.03232268999481039
c3136d3508ed0240b4b0409dcbcf3b19.jpeg
witch is : ZJL36 and min dis = -0.14764559376910635
9ac256d8e19b4a62d2a4b4fc056df81d.jpeg
witch is : ZJL240 and min dis = -0.08400134846330966
c3d5b3dd5d5e7e7831acc08b24712a9b.jpeg
witch is : ZJL88 and min dis = -0.0965860947600376
68318849a955995f7dfe7d2c35bbdb10.jpeg
witch is : ZJL214 and min dis = -0.1370400689223548
fbdd3eda07b60f87c88ea6dc03d5d9cc.jpeg
witch is : ZJL276 and min dis = -0.25063232458953005
2fee5cc995328eb94db9713f9525994e.jpeg
witch is : ZJL104 and min dis = -0.03077079827019863
affdc1753700ca61b95f6fcf5d20eaad.jpeg
witch is : ZJL36 and min dis = -0.12436880475226136
b246cf254345e80cb0d4c17e5d4062b6.jpeg
witch is : ZJL104 and min dis = -0.0009331370863328682


92b61c5683c979be257799c92e02f0d7.jpeg
witch is : ZJL41 and min dis = -0.16178807272492807
d977dcb0dfb275f678a7f01a8336e423.jpeg
witch is : ZJL220 and min dis = -0.0782597036514142
ff0f0b0833da82252a1f9592d237ecb5.jpeg
witch is : ZJL144 and min dis = -0.06794794453262883
6afbacdb78994e932681c14e1ba40c2d.jpeg
witch is : ZJL99 and min dis = -0.10157609946036725
true
f3222bc8fdff2a137719385619fa65aa.jpeg
witch is : ZJL174 and min dis = -0.18412998279865245
dcab686bf27ab7f742b325a8463c7da2.jpeg
witch is : ZJL207 and min dis = -0.2277430237206878
f047da94b843d2b820e7784c52ddf1e1.jpeg
witch is : ZJL212 and min dis = -0.19176819937909628
e57e2093dc588710710b68fb7aed2766.jpeg
witch is : ZJL238 and min dis = -0.15751753141110353
7d092ac2c208f2973f3ba9bd5e67735a.jpeg
witch is : ZJL220 and min dis = -0.09711276880512533
393c066c3b31ad109ace8acbd5eeac3c.jpeg
witch is : ZJL143 and min dis = 0.0035192247947029025
754ee7fc1ac3a04ee15e7be3333d14f6.jpeg
witch is : ZJL41 and min dis = -0.1347801698084968

4a44382c1bd843e1e25cf60baca68d35.jpeg
witch is : ZJL69 and min dis = -0.04231303117015145
true
daef375486f15d05cf57f8332d8c1a9a.jpeg
witch is : ZJL41 and min dis = -0.267417478141029
6d73c7c4a8f8288c247a490ee0def5f8.jpeg
witch is : ZJL38 and min dis = -0.06652460713376045
4025b1d71542c4a9509aedca0fc048d0.jpeg
witch is : ZJL104 and min dis = -0.160599759813113
8c743871ad31c39bf6f769bfac31812e.jpeg
witch is : ZJL240 and min dis = -0.20514683938529765
301617229eefc508e948251c168fa02c.jpeg
witch is : ZJL240 and min dis = -0.1893680794818146
af56674c1331008c8bacc12c39274a73.jpeg
witch is : ZJL104 and min dis = -0.14914907527330873
472aaf365eff607b40ceb345494078d4.jpeg
witch is : ZJL276 and min dis = -0.17192554011348468
5f78affa29a44854cb7a420c0c3f4e63.jpeg
witch is : ZJL41 and min dis = -0.10578388067613011
668a2f9432870ba661634585bf486b8b.jpeg
witch is : ZJL240 and min dis = -0.2098821063064962
2d5552a939bf8e20c8d8b921d2110f6b.jpeg
witch is : ZJL187 and min dis = -0.20391804988248508
3495

In [13]:
print(accury/length)

4